<a href="https://colab.research.google.com/github/irawan555/MachineLearningTasks/blob/main/13thWeekTasks/BAB%202%20/Irawan_Mardiansyah_1103210174_Fine_tuning_a_model_with_the_Trainer_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nama : Irawan Mardiansyah

NIM : 1103210174

Kelas : TK4504

Lecture : 13

# Processing the data (PyTorch)

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [15]:
!pip install datasets evaluate transformers[sentencepiece]

In [16]:
# Mengimpor fungsi yang diperlukan dari pustaka datasets
from datasets import load_dataset

# Mengimpor kelas-kelas yang diperlukan dari pustaka transformers
from transformers import AutoTokenizer, DataCollatorWithPadding

# Memuat dataset GLUE dengan metode "mrpc"
raw_datasets = load_dataset("glue", "mrpc")

# Memilih model BERT yang akan digunakan
checkpoint = "bert-base-uncased"

# Membuat tokenizer otomatis dari model yang dipilih
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# Mendefinisikan fungsi untuk tokenisasi setiap contoh dalam dataset
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

# Melakukan tokenisasi pada dataset menggunakan fungsi yang telah didefinisikan
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

# Membuat objek DataCollatorWithPadding untuk menyiapkan batch data dengan padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

Training

In [19]:
# Mengimpor kelas yang diperlukan dari pustaka Transformers
from transformers import AutoModelForSequenceClassification

# Membuat objek model untuk klasifikasi urutan (sequence)
# dari model yang sudah dilatih sebelumnya (pre-trained) dengan jumlah label 2
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Evaluation

In [21]:
{'accuracy': 0.8578431372549019, 'f1': 0.8996539792387542}

{'accuracy': 0.8578431372549019, 'f1': 0.8996539792387542}

In [22]:
# Mendefinisikan fungsi untuk menghitung metrik evaluasi
def compute_metrics(eval_preds):
    # Memuat metrik evaluasi dari pustaka 'evaluate' untuk tugas GLUE dan dataset MRPC
    metric = evaluate.load("glue", "mrpc")

    # Memisahkan logits (output model) dan label dari hasil evaluasi
    logits, labels = eval_preds

    # Menghasilkan prediksi dari logits dengan memilih indeks dengan nilai tertinggi
    predictions = np.argmax(logits, axis=-1)

    # Menghitung metrik menggunakan prediksi dan label
    return metric.compute(predictions=predictions, references=labels)